In [1]:
pip install scipy 

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from scipy.stats import norm

In [3]:
import datetime as dt

In [21]:
years = 5
enddate = dt.datetime.now()
startdate = enddate - dt.timedelta(days = 365*years)

In [22]:
tickers = ['AAPL','MSFT','TSLA','GOOGL','NVDA']

In [23]:
adj_close = pd.DataFrame()

for ticker in tickers:
    data = yf.download(ticker, start = startdate, end = enddate)
    adj_close[ticker] = data['Adj Close'] 

print(adj_close)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

                  AAPL        MSFT        TSLA       GOOGL        NVDA
Date                                                                  
2020-01-28   76.998711  158.260101   37.793331   72.263039    6.173369
2020-01-29   78.610489  160.727829   38.732666   72.571922    6.112873
2020-01-30   78.496574  165.261566   42.720669   72.449860    6.119595
2020-01-31   75.016113  162.822479   43.371334   71.380241    5.886075
2020-02-03   74.810112  166.791946   52.000000   73.862244    5.983167
...                ...         ...         ...         ...         ...
2025-01-17  229.979996  429.029999  426.500000  196.000000  137.710007
2025-01-21  222.639999  428.500000  424.070007  198.050003  140.830002
2025-01-22  223.830002  446.200012  415.109985  198.369995  147.070007
2025-01-23  223.660004  446.709991  412.380005  197.979996  147.220001
2025-01-24  222.779999  444.059998  406.579987  200.210007  142.619995

[1256 rows x 5 columns]


In [24]:
log_return = np.log(adj_close/adj_close.shift(1))
log_return = log_return.dropna()
print(log_return)

                AAPL      MSFT      TSLA     GOOGL      NVDA
Date                                                        
2020-01-29  0.020716  0.015473  0.024551  0.004265 -0.009848
2020-01-30 -0.001450  0.027817  0.098000 -0.001683  0.001099
2020-01-31 -0.045352 -0.014869  0.015116 -0.014874 -0.038907
2020-02-03 -0.002750  0.024087  0.181445  0.034181  0.016361
2020-02-04  0.032481  0.032386  0.128619 -0.025404  0.027902
...              ...       ...       ...       ...       ...
2025-01-17  0.007507  0.010426  0.030181  0.015891  0.030524
2025-01-21 -0.032436 -0.001236 -0.005714  0.010405  0.022403
2025-01-22  0.005331  0.040477 -0.021355  0.001614  0.043355
2025-01-23 -0.000760  0.001142 -0.006598 -0.001968  0.001019
2025-01-24 -0.003942 -0.005950 -0.014165  0.011201 -0.031744

[1255 rows x 5 columns]


In [25]:
port_val = 1000000
weight = np.array([1/len(tickers)]*len(tickers))
print(weight)

[0.2 0.2 0.2 0.2 0.2]


In [30]:
hist_return = (log_return * weight).sum(axis =1)
hist_return

Date
2020-01-29    0.011031
2020-01-30    0.024756
2020-01-31   -0.019777
2020-02-03    0.050665
2020-02-04    0.039197
                ...   
2025-01-17    0.018906
2025-01-21   -0.001316
2025-01-22    0.013884
2025-01-23   -0.001433
2025-01-24   -0.008920
Length: 1255, dtype: float64

In [33]:
days = 5
range_ret = hist_return.rolling(window = days).sum()
range_ret = range_ret.dropna()
range_ret

Date
2020-02-04    0.105872
2020-02-05    0.061535
2020-02-06    0.053914
2020-02-07    0.069321
2020-02-10    0.043683
                ...   
2025-01-17    0.021059
2025-01-21    0.023428
2025-01-22    0.046114
2025-01-23    0.007463
2025-01-24    0.021122
Length: 1251, dtype: float64

In [35]:
cf = 0.95
var = np.percentile(range_ret,100 - (cf*100))
var_dollar = var * port_val
var_dollar

-68175.84809583446